# Travel Customer Support with Handoff Orchestration designed by Jason

This notebook demonstrates **handoff orchestration** using the Microsoft Agent Framework. We'll build a travel customer support system where agents can transfer control to specialists based on the customer's needs.

## What You'll Learn:
1. **Handoff Orchestration**: Dynamic agent routing based on context and expertise
2. **HandoffBuilder**: High-level API for building handoff workflows
3. **Specialist Routing**: Agents can hand off to other agents dynamically
4. **Multi-turn Conversations**: Seamless context preservation across handoffs
5. **Customer Support Flow**: Real-world application of agent handoffs

## Prerequisites:
- Microsoft Agent Framework installed
- Azure subscription with AI Foundry project and Azure OpenAI resources
- Azure CLI installed and authenticated (`az login`)
- Understanding of basic agent concepts

In [1]:
import asyncio
import json
import os
from collections.abc import AsyncIterable
from typing import Any, cast

from agent_framework import (
    ChatMessage,
    HandoffBuilder,
    HandoffUserInputRequest,
    RequestInfoEvent,
    WorkflowEvent,
    WorkflowOutputEvent,
    WorkflowRunState,
    WorkflowStatusEvent,
)

# GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

# Azure AI imports for Foundry support
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

## Step 1: Define Pydantic Models for Structured Outputs

These models define the schema that each specialized agent will return. This ensures consistent and parseable responses from all agents.

In [2]:
class FlightBookingResult(BaseModel):
    """Flight booking confirmation from the booking agent."""
    destination: str
    departure_date: str
    return_date: str
    flight_booking_reference: str
    passenger_name: str
    flight_details: str
    total_cost: str
    status: str

class HotelBookingResult(BaseModel):
    """Hotel booking confirmation from the hotel booking agent."""
    destination: str
    check_in_date: str
    check_out_date: str
    hotel_booking_reference: str
    passenger_name: str
    hotel_details: str
    total_cost: str
    status: str

class CarBookingResult(BaseModel):
    """Car Rental booking confirmation from the car rental agent."""
    destination: str
    check_in_date: str
    check_out_date: str
    car_booking_reference: str
    passenger_name: str
    car_details: str
    total_cost: str
    status: str
    
class TripCheckResult(BaseModel):
    """Trip confirmation result from the trip check agent."""
    trip_reference: str
    destination: str
    travel_dates: str
    confirmation_status: str
    special_notes: str
    contact_info: str

## Step 2: Load Environment Variables & Initialize Client

This notebook uses **Azure AI Foundry** with token-based authentication via DefaultAzureCredential.

In [3]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"),
    api_key=os.environ.get("GITHUB_TOKEN"),
    model_id="gpt-4.1-mini"
)

""" # Load environment variables
load_dotenv()

# Initialize Azure OpenAI chat client with required parameters
chat_client = AzureOpenAIChatClient(
    credential=DefaultAzureCredential(),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
)

# Initialize AI Project client for Azure AI Foundry support
project_client = AIProjectClient(
    endpoint=os.getenv("PROJECT_ENDPOINT", ""),
    credential=DefaultAzureCredential()
) """

' # Load environment variables\nload_dotenv()\n\n# Initialize Azure OpenAI chat client with required parameters\nchat_client = AzureOpenAIChatClient(\n    credential=DefaultAzureCredential(),\n    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),\n    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")\n)\n\n# Initialize AI Project client for Azure AI Foundry support\nproject_client = AIProjectClient(\n    endpoint=os.getenv("PROJECT_ENDPOINT", ""),\n    credential=DefaultAzureCredential()\n) '

### Configuration Requirements

**Azure AI Foundry with Token-Based Authentication**

This notebook uses Azure AD authentication via `DefaultAzureCredential` (no API keys required).

**Required Environment Variables:**
- `PROJECT_ENDPOINT` - Azure AI Project endpoint
- `AZURE_OPENAI_ENDPOINT` - Azure OpenAI endpoint  
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME` - Model deployment name (e.g., gpt-4o)

**Setup Steps:**
1. Run `az login` to authenticate with Azure
2. Add environment variables to your `.env` file:
   ```
   PROJECT_ENDPOINT=https://your-project.api.azureml.ms
   AZURE_OPENAI_ENDPOINT=https://your-openai.openai.azure.com
   AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4o
   ```
3. Ensure your Azure account has appropriate permissions for the AI Project and OpenAI resources

**Authentication:**
- Uses `DefaultAzureCredential` for Azure AD authentication
- Uses `get_bearer_token_provider` for OpenAI token generation
- No API keys required (ideal for tenants with API keys disabled)

## Step 3: Create Four Specialized Travel Support Agents

Each agent has specific expertise and can hand off to appropriate specialists based on customer needs.

In [4]:
# Agent 1: Travel Agent (Main triage agent)
travel_agent = chat_client.create_agent(
    instructions=(
        "You are a friendly travel agent for a travel company. "
        "Assess customer requests and route them to the appropriate specialist: "
        "- For flight bookings or reservations: call handoff_to_flight_booking_agent "
        "- For hotel bookings or reservations: call handoff_to_hotel_booking_agent "
        "- For car rental bookings or reservations: call handoff_to_car_booking_agent "
        "- For trip confirmations or travel plan checks: call handoff_to_trip_check_agent "
        "Be welcoming and ensure customers feel heard before routing them."
    ),
    name="travel_agent",
)


# Agent 2: Flight Booking Agent (Flight booking specialist)
flight_booking_agent = chat_client.create_agent(
    instructions=(
        "You are a flight booking specialist. Handle all flight reservations and bookings. "
        "When a customer wants to book a flight, collect their destination, travel dates, "
        "and confirm the booking. Always provide a flight booking reference number. "
        "Return structured JSON with booking details. "
        "The flight is always confirmed as booked regardless of destination."
    ),
    name="flight_booking_agent",
    response_format=FlightBookingResult,
)

# Agent 3: Hotel Booking Agent (Hotel booking specialist)
hotel_booking_agent = chat_client.create_agent(
    instructions=(
        "You are a hotel booking specialist. Handle all hotel reservations and bookings. "
        "When a customer wants to book a hotel, collect their destination, travel dates, "
        "and confirm the booking. Always provide a hotel booking reference number. "
        "Return structured JSON with booking details. "
        "The hotel is always confirmed as booked regardless of destination."
    ),
    name="hotel_booking_agent",
    response_format=HotelBookingResult,
)

# Agent 4: Car Booking Agent (Car rental specialist)
car_booking_agent = chat_client.create_agent(
    instructions=(
        "You are a car rental specialist. Handle all car rental reservations and bookings. "
        "When a customer wants to book a car, collect their destination, travel dates, "
        "and confirm the booking. Always provide a car booking reference number. "
        "Return structured JSON with booking details. "
        "The car rental is always confirmed as booked regardless of destination."
    ),
    name="car_booking_agent",
    response_format=CarBookingResult,
)

# Agent 5: Trip Check Agent (Travel confirmation specialist)
trip_check_agent = chat_client.create_agent(
    instructions=(
        "You are a travel confirmation specialist. Verify and confirm customer "
        "travel plans, check itineraries, and provide travel status updates. "
        "Always confirm that travel plans are in order and provide reassurance. "
        "Return structured JSON with confirmation details. "
        "All travel plans are confirmed as valid and ready."
    ),
    name="trip_check_agent",
    response_format=TripCheckResult,
)




## Step 4: Build the Handoff Workflow

The HandoffBuilder creates a workflow where the customer support agent can dynamically hand off to specialists based on customer needs.


In [5]:
workflow = (
    HandoffBuilder(
        name="travel_agent_handoff",
        participants=[travel_agent, flight_booking_agent, hotel_booking_agent, car_booking_agent, trip_check_agent],
    )
    .set_coordinator(travel_agent)  # Main agent that receives initial requests
    .add_handoff(travel_agent, [flight_booking_agent, hotel_booking_agent, car_booking_agent, trip_check_agent])
    .with_termination_condition(
        lambda conv: sum(1 for msg in conv if msg.role.value == "user") > 3
    )  # Stop after 3 user messages
    .build()
)

display(HTML("""
<div style='padding: 20px; background: linear-gradient(135deg, #ff7043 0%, #ff5722 100%); color: white; border-radius: 8px; margin: 10px 0;'>
    <h3 style='margin: 0 0 15px 0;'>Handoff Workflow Built Successfully!</h3>
    <p style='margin: 0; line-height: 1.6;'>
        <strong>Handoff Flow:</strong><br>
        • User Request → <strong>Travel Agent</strong> (triage)<br>
        • Support Agent → <strong>Specialist Agent</strong> (dynamic handoff)<br>
        • Specialist → <strong>Resolution</strong> (expert handling)<br>
        • System → <strong>User Response</strong> (final result)
    </p>
</div>
"""))



## Step 5: Helper Functions for Event Processing

These functions help us process workflow events and handle user input requests during the handoff process.

In [6]:
async def drain_events(stream: AsyncIterable[WorkflowEvent]) -> list[WorkflowEvent]:
    """Collect all events from an async stream into a list."""
    return [event async for event in stream]


def handle_workflow_events(events: list[WorkflowEvent]) -> list[RequestInfoEvent]:
    """Process workflow events and extract pending user input requests."""
    requests: list[RequestInfoEvent] = []
    
    for event in events:
        if isinstance(event, WorkflowStatusEvent) and event.state in {
            WorkflowRunState.IDLE,
            WorkflowRunState.IDLE_WITH_PENDING_REQUESTS,
        }:
           print(f"[Workflow Status] {event.state.name}")

        elif isinstance(event, WorkflowOutputEvent):
            conversation = cast(list[ChatMessage], event.data)
            if isinstance(conversation, list):
                print("\n=== Final Conversation ===")
                for message in conversation:
                    # Filter out messages with no text (tool calls)
                    if not message.text.strip():
                        continue
                    speaker = message.author_name or message.role.value
                    print(f"- {speaker}: {message.text}")
                print("==========================")

        elif isinstance(event, RequestInfoEvent):
            if isinstance(event.data, HandoffUserInputRequest):
                print_handoff_request(event.data)
                requests.append(event)

    return requests


def print_handoff_request(request: HandoffUserInputRequest) -> None:
    """Display a user input request with conversation context."""
    print("\n=== User Input Requested ===")
    # Filter out messages with no text for cleaner display
    messages_with_text = [
        msg for msg in request.conversation if msg.text.strip()]
    print(f"Last {len(messages_with_text)} messages in conversation:")
    for message in messages_with_text[-3:]:  # Show last 3 for brevity
        speaker = message.author_name or message.role.value
        text = message.text[:100] + \
            "..." if len(message.text) > 100 else message.text
        print(f"  {speaker}: {text}")
    print("============================")


print("Helper functions defined for event processing")

Helper functions defined for event processing


## Step 6: Test Case 1 - Flight Booking Request

Let's test our handoff workflow with a flight booking request. The customer support agent should hand off to the booking agent.


In [7]:
async def test_flight_booking_handoff():
    """Test handoff workflow for flight booking requests."""

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Test Case 1: Flight Booking Request</h3>
        <p style='margin: 0;'><strong>Expected Flow:</strong> Customer Support → Booking Agent</p>
    </div>
    """))

    # Start the workflow
    print("[User]: I want to book a flight to Paris for next month")
    events = await drain_events(
        workflow.run_stream("I want to book a flight to Paris for next month")
    )
    pending_requests = handle_workflow_events(events)

    # Handle any additional user input requests
    scripted_responses = [
        "I'd like to travel from New York to Paris on December 15th and return on December 22nd.",
        "Yes, please confirm the booking under the name John Smith."
    ]

    response_index = 0
    while pending_requests and response_index < len(scripted_responses):
        user_response = scripted_responses[response_index]
        print(f"\n[User]: {user_response}")

        responses = {req.request_id: user_response for req in pending_requests}
        events = await drain_events(workflow.send_responses_streaming(responses))
        pending_requests = handle_workflow_events(events)

        response_index += 1

    # Extract and display the final booking result
    if events:
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "flight_booking_agent" and message.text.strip():
                        try:
                            flight_booking_data = FlightBookingResult.model_validate_json(
                                message.text)
                            display_flight_booking_result(flight_booking_data)
                        except Exception as e:
                            print(f"Could not parse booking result: {e}")


def display_flight_booking_result(flightbooking: FlightBookingResult):
    """Display flight booking result in a formatted section."""

    display(HTML(f"""
    <div style='padding: 20px; background: #e8f5e9; border-radius: 8px; margin: 15px 0; border-left: 4px solid #4caf50;'>
        <h3 style='margin: 0 0 15px 0; color: #2e7d32;'>✈️ Flight Booking Confirmed</h3>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
            <div>
                <strong style='color: #333;'>Flight Booking Reference:</strong> {flightbooking.flight_booking_reference}<br>
                <strong style='color: #333;'>Passenger:</strong> {flightbooking.passenger_name}<br>
                <strong style='color: #333;'>Status:</strong> <span style='color: #4caf50; font-weight: bold;'>{flightbooking.status}</span>
            </div>
            <div>
                <strong style='color: #333;'>Destination:</strong> {flightbooking.destination}<br>
                <strong style='color: #333;'>Total Cost:</strong> {flightbooking.total_cost}<br>
                <strong style='color: #333;'>Departure:</strong> {flightbooking.departure_date}
            </div>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Flight Details:</strong> {flightbooking.flight_details}
        </div>
        <div style='background: rgba(76,175,80,0.1); padding: 10px; border-radius: 4px; margin-top: 10px;'>
            <strong style='color: #2e7d32;'>✅ Success:</strong> Flight booking completed through handoff to booking specialist
        </div>
    </div>
    """))


# Run the booking test
await test_flight_booking_handoff()

[User]: I want to book a flight to Paris for next month

=== User Input Requested ===
Last 3 messages in conversation:
  user: I want to book a flight to Paris for next month
  travel_agent: Thank you for reaching out! I'd be happy to assist you with booking your flight to Paris for next mo...
  flight_booking_agent: {"departure_date":"2024-07-15","destination":"Paris","flight_booking_reference":"PARIS20240715XK9","...
[Workflow Status] IDLE_WITH_PENDING_REQUESTS

[User]: I'd like to travel from New York to Paris on December 15th and return on December 22nd.

=== User Input Requested ===
Last 6 messages in conversation:
  user: I'd like to travel from New York to Paris on December 15th and return on December 22nd.
  travel_agent: Thank you for sharing your travel dates and locations! I'll connect you with our flight booking spec...
  flight_booking_agent: {"departure_date":"2024-12-15","destination":"Paris","flight_booking_reference":"NYC2PARIS20241215XZ...
[Workflow Status] IDLE_WITH_

## Step 7: Test Case 2 - Hotel Booking Request

Let's test our handoff workflow with a Hotel booking request. The customer support agent should hand off to the Hotel agent.


In [7]:
async def test_hotel_booking_handoff():
    """Test handoff workflow for hotel booking requests."""

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Test Case 1: Hotel Booking Request</h3>
        <p style='margin: 0;'><strong>Expected Flow:</strong> Travel Agent  → Hotel Booking Agent</p>
    </div>
    """))

    # Start the workflow
    print("[User]: I want to make a reservation for a hotel in Paris next month")
    events = await drain_events(
        workflow.run_stream("I want to make a reservation for a hotel in Paris next month")
    )
    pending_requests = handle_workflow_events(events)

    # Handle any additional user input requests
    scripted_responses = [
        "I'd like to stay at a hotel in Paris from December 15th to December 22nd.",
        "Yes, please confirm the booking under the name John Smith."
    ]

    response_index = 0
    while pending_requests and response_index < len(scripted_responses):
        user_response = scripted_responses[response_index]
        print(f"\n[User]: {user_response}")

        responses = {req.request_id: user_response for req in pending_requests}
        events = await drain_events(workflow.send_responses_streaming(responses))
        pending_requests = handle_workflow_events(events)

        response_index += 1

    # Extract and display the final Hotel booking result
    if events:
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "hotel_booking_agent" and message.text.strip():
                        try:
                            hotel_booking_data = HotelBookingResult.model_validate_json(
                                message.text)
                            display_hotel_booking_result(hotel_booking_data)
                        except Exception as e:
                            print(f"Could not parse Hotel booking result: {e}")


def display_hotel_booking_result(hotelbooking: HotelBookingResult):
    """Display hotel booking result in a formatted section."""

    display(HTML(f"""
    <div style='padding: 20px; background: #e8f5e9; border-radius: 8px; margin: 15px 0; border-left: 4px solid #4caf50;'>
        <h3 style='margin: 0 0 15px 0; color: #2e7d32;'>🏨 Hotel Booking Confirmed</h3>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
            <div>
                <strong style='color: #333;'>Hotel Booking Reference:</strong> {hotelbooking.hotel_booking_reference}<br>
                <strong style='color: #333;'>Guest Name:</strong> {hotelbooking.passenger_name}<br>
                <strong style='color: #333;'>Status:</strong> <span style='color: #4caf50; font-weight: bold;'>{hotelbooking.status}</span>
            </div>
            <div>
                <strong style='color: #333;'>Destination:</strong> {hotelbooking.destination}<br>
                <strong style='color: #333;'>Total Cost:</strong> {hotelbooking.total_cost}<br>
                <strong style='color: #333;'>Check-In Date:</strong> {hotelbooking.check_in_date}<br>
                <strong style='color: #333;'>Check-Out Date:</strong> {hotelbooking.check_out_date}
            </div>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Hotel Details:</strong> {hotelbooking.hotel_details}
        </div>
        <div style='background: rgba(76,175,80,0.1); padding: 10px; border-radius: 4px; margin-top: 10px;'>
            <strong style='color: #2e7d32;'>✅ Success:</strong> Hotel booking completed through handoff to booking specialist
        </div>
    </div>
    """))


# Run the booking test
await test_hotel_booking_handoff()

[User]: I want to make a reservation for a hotel in Paris next month

=== User Input Requested ===
Last 3 messages in conversation:
  user: I want to make a reservation for a hotel in Paris next month
  travel_agent: That sounds like a wonderful trip! I'd be happy to assist you with your hotel reservation in Paris n...
  hotel_booking_agent: {"check_in_date":"2024-07-10","check_out_date":"2024-07-15","destination":"Paris","hotel_booking_ref...
[Workflow Status] IDLE_WITH_PENDING_REQUESTS

[User]: I'd like to stay at a hotel in Paris from December 15th to December 22nd.

=== User Input Requested ===
Last 5 messages in conversation:
  hotel_booking_agent: {"check_in_date":"2024-07-10","check_out_date":"2024-07-15","destination":"Paris","hotel_booking_ref...
  user: I'd like to stay at a hotel in Paris from December 15th to December 22nd.
  hotel_booking_agent: {"check_in_date":"2024-12-15","check_out_date":"2024-12-22","destination":"Paris","hotel_booking_ref...
[Workflow Status] IDLE_WI

## Step 8 Test Case 3 - Car Booking Request

Let's test our handoff workflow with a Car booking request. The customer support agent should hand off to the Car agent.

In [7]:
async def test_car_booking_handoff():
    """Test handoff workflow for car booking requests."""

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Test Case 1: Car Booking Request</h3>
        <p style='margin: 0;'><strong>Expected Flow:</strong> Travel Agent  → Car Booking Agent</p>
    </div>
    """))

    # Start the workflow
    print("[User]: I want to book a car in Paris for next month")
    events = await drain_events(
        workflow.run_stream("I want to book a car in Paris for next month")
    )
    pending_requests = handle_workflow_events(events)

    # Handle any additional user input requests
    scripted_responses = [
        "I'd like to rent a car in Paris from December 15th to December 22nd.",
        "Yes, please confirm the booking under the name John Smith."
    ]

    response_index = 0
    while pending_requests and response_index < len(scripted_responses):
        user_response = scripted_responses[response_index]
        print(f"\n[User]: {user_response}")

        responses = {req.request_id: user_response for req in pending_requests}
        events = await drain_events(workflow.send_responses_streaming(responses))
        pending_requests = handle_workflow_events(events)

        response_index += 1

    # Extract and display the final Car booking result
    if events:
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "car_booking_agent" and message.text.strip():
                        try:
                            car_booking_data = CarBookingResult.model_validate_json(
                                message.text)
                            display_car_booking_result(car_booking_data)
                        except Exception as e:
                            print(f"Could not parse Car booking result: {e}")


def display_car_booking_result(carbooking: CarBookingResult):
    """Display car booking result in a formatted section."""

    display(HTML(f"""
    <div style='padding: 20px; background: #e8f5e9; border-radius: 8px; margin: 15px 0; border-left: 4px solid #4caf50;'>
        <h3 style='margin: 0 0 15px 0; color: #2e7d32;'>🚗 Car Booking Confirmed</h3>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
            <div>
                <strong style='color: #333;'>Car Booking Reference:</strong> {carbooking.car_booking_reference}<br>
                <strong style='color: #333;'>Renter Name:</strong> {carbooking.passenger_name}<br>
                <strong style='color: #333;'>Status:</strong> <span style='color: #4caf50; font-weight: bold;'>{carbooking.status}</span>
            </div>
            <div>
                <strong style='color: #333;'>Destination:</strong> {carbooking.destination}<br>
                <strong style='color: #333;'>Total Cost:</strong> {carbooking.total_cost}<br>
                <strong style='color: #333;'>Check-In Date:</strong> {carbooking.check_in_date}<br>
                <strong style='color: #333;'>Check-Out Date:</strong> {carbooking.check_out_date}
            </div>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Car Details:</strong> {carbooking.car_details}
        </div>
        <div style='background: rgba(76,175,80,0.1); padding: 10px; border-radius: 4px; margin-top: 10px;'>
            <strong style='color: #2e7d32;'>✅ Success:</strong> Car booking completed through handoff to booking specialist
        </div>
    </div>
    """))


# Run the booking test
await test_car_booking_handoff()

[User]: I want to book a car in Paris for next month

=== User Input Requested ===
Last 3 messages in conversation:
  user: I want to book a car in Paris for next month
  travel_agent: Thank you for sharing your travel plans! Booking a car in Paris sounds like a great way to explore t...
  car_booking_agent: {"car_booking_reference":"PAR123456","car_details":"Compact Sedan","check_in_date":"2024-07-01","che...
[Workflow Status] IDLE_WITH_PENDING_REQUESTS

[User]: I'd like to rent a car in Paris from December 15th to December 22nd.

=== User Input Requested ===
Last 6 messages in conversation:
  user: I'd like to rent a car in Paris from December 15th to December 22nd.
  travel_agent: I see you would like to rent a car in Paris from December 15th to December 22nd. I will connect you ...
  car_booking_agent: {"car_booking_reference":"PAR789012","car_details":"Standard SUV","check_in_date":"2024-12-15","chec...
[Workflow Status] IDLE_WITH_PENDING_REQUESTS

[User]: Yes, please confirm the 

## Step 9: Test Case 4 - Trip Confirmation Request

Let's test our handoff workflow with a trip confirmation request. The customer support agent should hand off to the trip check agent.

In [8]:
async def test_trip_check_handoff():
    """Test handoff workflow for trip confirmation requests."""

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Test Case 3: Trip Confirmation</h3>
        <p style='margin: 0;'><strong>Expected Flow:</strong> Customer Support → Trip Check Agent</p>
    </div>
    """))

    # Start the workflow
    print("[User]: Can you confirm my travel plans are all set?")
    events = await drain_events(
        workflow.run_stream("Can you confirm my travel plans are all set?")
    )
    pending_requests = handle_workflow_events(events)

    # Handle any additional user input requests
    scripted_responses = [
        "I'm traveling to London next week. My confirmation number is TR98765.",
        "Perfect, thank you for checking everything is ready!"
    ]

    response_index = 0
    while pending_requests and response_index < len(scripted_responses):
        user_response = scripted_responses[response_index]
        print(f"\n[User]: {user_response}")
        
        responses = {req.request_id: user_response for req in pending_requests}
        events = await drain_events(workflow.send_responses_streaming(responses))
        pending_requests = handle_workflow_events(events)
        
        response_index += 1
    # Extract and display the final trip check result
    if events:
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "trip_check_agent" and message.text.strip():
                        try:
                            trip_data = TripCheckResult.model_validate_json(
                                message.text)
                            display_trip_check_result(trip_data)
                        except Exception as e:
                            print(f"Could not parse trip check result: {e}")


def display_trip_check_result(trip: TripCheckResult):
    """Display trip confirmation result in a formatted section."""

    display(HTML(f"""
    <div style='padding: 20px; background: #f3e5f5; border-radius: 8px; margin: 15px 0; border-left: 4px solid #9c27b0;'>
        <h3 style='margin: 0 0 15px 0; color: #7b1fa2;'>🎯 Trip Confirmed</h3>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
            <div>
                <strong style='color: #333;'>Trip Reference:</strong> {trip.trip_reference}<br>
                <strong style='color: #333;'>Destination:</strong> {trip.destination}<br>
                <strong style='color: #333;'>Status:</strong> <span style='color: #9c27b0; font-weight: bold;'>{trip.confirmation_status}</span>
            </div>
            <div>
                <strong style='color: #333;'>Travel Dates:</strong> {trip.travel_dates}<br>
                <strong style='color: #333;'>Contact Info:</strong> {trip.contact_info}
            </div>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Special Notes:</strong> {trip.special_notes}
        </div>
        <div style='background: rgba(156,39,176,0.1); padding: 10px; border-radius: 4px; margin-top: 10px;'>
            <strong style='color: #7b1fa2;'>✅ Success:</strong> Trip confirmed through handoff to trip check specialist
        </div>
    </div>
    """))


# Run the trip check test
await test_trip_check_handoff()

   

[User]: Can you confirm my travel plans are all set?

=== User Input Requested ===
Last 10 messages in conversation:
  travel_agent: Thank you for providing the name, John Smith. I will make sure your car rental booking in Paris from...
  car_booking_agent: {"car_booking_reference":"PAR789012","car_details":"Standard SUV","check_in_date":"2024-12-15","chec...
  travel_agent: I would be happy to help you confirm your travel plans. To assist you best, could you please specify...
[Workflow Status] IDLE_WITH_PENDING_REQUESTS

[User]: I'm traveling to London next week. My confirmation number is TR98765.

=== Final Conversation ===
- user: I want to book a car in Paris for next month
- travel_agent: Thank you for sharing your travel plans! Booking a car in Paris sounds like a great way to explore the city and its surroundings. I'll connect you with our car rental specialist who can assist you with the booking. One moment please.
- car_booking_agent: {"car_booking_reference":"PAR123456","car_

## Step 10: Workflow Analysis - Understanding Handoff Flow


In [14]:
async def analyze_handoff_patterns():
    """Analyze different handoff patterns and routing decisions."""

    display(HTML("""
    <div style='padding: 20px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #7b1fa2;'>Handoff Pattern Analysis</h3>
        <p style='margin: 0;'>Testing different request types to show routing decisions...</p>
    </div>
    """))

    test_requests = [
        "I want to book a round-trip flight to Tokyo",
        "I want to book a hotel in the same city",
        "I want to book a car in the same city",
        "Please check if my travel itinerary is confirmed",
        "Can you share the itinerary details with me?",
    ]

    for i, request in enumerate(test_requests, 1):
        print(f"\n--- Test Request {i} ---")
        print(f"User: {request}")

        # Run workflow and capture routing decision
        events = await drain_events(workflow.run_stream(request))

        # Analyze which agent was activated
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "travel_agent":
                        print(f"Support Agent: {message.text[:100]}...")
                    elif message.author_name in ["flight_booking_agent","hotel_booking_agent" ,"car_booking_agent", "trip_check_agent"]:
                        agent_type = {
                            "flight_booking_agent": "🛫 FLIGHT BOOKING SPECIALIST",
                            "hotel_booking_agent": "🏨 HOTEL BOOKING SPECIALIST",
                            "car_booking_agent": "🚗 CAR BOOKING SPECIALIST",
                            "trip_check_agent": "🎯 TRIP CHECK SPECIALIST"
                        }[message.author_name]
                        print(f"Routed to: {agent_type}")
                        break
                break
    display(HTML("""
    <div style='padding: 25px; background: linear-gradient(135deg, #9c27b0 0%, #673ab7 100%); color: white; border-radius: 12px; 
                box-shadow: 0 4px 12px rgba(156,39,176,0.4); margin: 20px 0;'>
        <h2 style='margin: 0 0 20px 0;'>Handoff Analysis Results</h2>
        <div style='background: rgba(255,255,255,0.15); padding: 15px; border-radius: 8px;'>
            <h4 style='margin: 0 0 10px 0;'>Key Observations</h4>
            <ul style='margin: 0; padding-left: 20px; line-height: 1.6;'>
                <li><strong>Dynamic Routing:</strong> Travel Agent analyzes request intent</li>
                <li><strong>Context Preservation:</strong> Full conversation history maintained</li>
                <li><strong>Specialist Focus:</strong> Each agent handles their expertise area</li>
                <li><strong>Seamless Handoff:</strong> Users don't need to repeat information</li>
            </ul>
        </div>
    </div>
    """))

    # Run the analysis
await analyze_handoff_patterns()


--- Test Request 1 ---
User: I want to book a round-trip flight to Tokyo

--- Test Request 2 ---
User: I want to book a hotel in the same city

--- Test Request 3 ---
User: I want to book a car in the same city

--- Test Request 4 ---
User: Please check if my travel itinerary is confirmed

--- Test Request 5 ---
User: Can you share the itinerary details with me?
